# 🔍 Avaliação do Detector PIIGuardian

Este notebook demonstra a avaliação completa do detector de dados pessoais,
incluindo métricas de recall, precisão e análise de falsos negativos.

## Critérios do Hackathon
Em caso de empate:
1. **Menor número de falsos negativos** ← Critério principal
2. Menor número de falsos positivos
3. Maior nota P1

In [ ]:
# Imports
import sys
sys.path.insert(0, '..')

import json
import pandas as pd
from src.detector import PIIGuardian
from src.utils import normalize_text

# Inicializa detector
detector = PIIGuardian(mode='balanced')
print("✓ Detector inicializado")

In [ ]:
# Carrega dados de teste
with open('../data/sample_pedidos.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

test_cases = data['data']
print(f"Total de casos de teste: {len(test_cases)}")

In [ ]:
# Executa detecção em todos os casos
results = []

for case in test_cases:
    result = detector.detect(normalize_text(case['text']))
    
    results.append({
        'id': case['id'],
        'text': case['text'][:50] + '...',
        'expected_pii': case['expected_has_pii'],
        'detected_pii': result.has_pii,
        'expected_count': len(case['expected_entities']),
        'detected_count': len(result.entities),
        'correct': case['expected_has_pii'] == result.has_pii,
        'time_ms': result.metadata['processing_time_ms']
    })

df_results = pd.DataFrame(results)
df_results

In [ ]:
# Calcula métricas
tp = len(df_results[(df_results['expected_pii'] == True) & (df_results['detected_pii'] == True)])
tn = len(df_results[(df_results['expected_pii'] == False) & (df_results['detected_pii'] == False)])
fp = len(df_results[(df_results['expected_pii'] == False) & (df_results['detected_pii'] == True)])
fn = len(df_results[(df_results['expected_pii'] == True) & (df_results['detected_pii'] == False)])

precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
accuracy = (tp + tn) / len(df_results)

print("=" * 50)
print("MÉTRICAS DE AVALIAÇÃO")
print("=" * 50)
print(f"\nMatriz de Confusão:")
print(f"  TP (Verdadeiros Positivos): {tp}")
print(f"  TN (Verdadeiros Negativos): {tn}")
print(f"  FP (Falsos Positivos): {fp}")
print(f"  FN (Falsos Negativos): {fn} ⚠️")
print(f"\nMétricas:")
print(f"  Precisão: {precision:.4f} ({precision*100:.2f}%)")
print(f"  Recall: {recall:.4f} ({recall*100:.2f}%)")
print(f"  F1-Score: {f1:.4f} ({f1*100:.2f}%)")
print(f"  Acurácia: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"\nTempo médio: {df_results['time_ms'].mean():.2f}ms")

In [ ]:
# Análise de falsos negativos (crítico!)
false_negatives = df_results[(df_results['expected_pii'] == True) & (df_results['detected_pii'] == False)]

if len(false_negatives) > 0:
    print("⚠️ FALSOS NEGATIVOS ENCONTRADOS:")
    for _, row in false_negatives.iterrows():
        print(f"  ID: {row['id']} - {row['text']}")
else:
    print("✅ ZERO FALSOS NEGATIVOS! Excelente!")

In [ ]:
# Teste detalhado em um exemplo
exemplo = "Meu CPF é 123.456.789-09 e telefone (61) 99999-8888"
resultado = detector.detect(exemplo)

print(f"Texto: {exemplo}")
print(f"\nResultado:")
print(f"  Contém PII: {resultado.has_pii}")
print(f"  Entidades detectadas:")
for e in resultado.entities:
    print(f"    - {e.type}: '{e.value}' (conf: {e.confidence:.0%})")

## Conclusões

O detector PIIGuardian demonstra:
- **Alto recall** - minimiza falsos negativos (critério de desempate)
- **Boa precisão** - poucos falsos positivos
- **Performance rápida** - processamento em milissegundos